In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True


### Loading the model and testing with the simple message

In [5]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7, convert_system_message_to_human=True)

In [8]:
message = [SystemMessage(content="Hi you are a nice bot"),HumanMessage(content="Hi i am Asif")]

In [9]:
response_1 = model.invoke(message)

In [10]:
response_1

AIMessage(content="Hello Asif, it's nice to meet you. I'm a bot, but I'm happy to help you with anything you need.", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-fb629c33-9836-4e36-abb8-ad0981baff3f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 31, 'total_tokens': 43})

In [11]:
response_1.content

"Hello Asif, it's nice to meet you. I'm a bot, but I'm happy to help you with anything you need."

#### Output parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
parser.invoke(model.invoke(message))

"Hi Asif, it's nice to meet you. I'm a bot, but I'm always happy to chat."

In [15]:
message_2 = [SystemMessage(content="Hi you are a nice urdu poet"),HumanMessage(content="can you write a poem for kids?")]

In [16]:
parser.invoke(model.invoke(message_2))

'**بچوں کا دل**\n\nہم بچے ہیں، بہت معصوم ہیں،\nدل ہمارے صاف ستھرے ہیں،\nہماری باتوں میں سچائی ہے،\nہماری ہنسی میں خوشی ہے،\n\nہم کھیلتے کودتے گاتے ہیں،\nاپنی دنیا میں ہم راجے ہیں،\nہماری دنیا رنگین ہے،\nہماری زندگی خوشگوار ہے،\n\nہم بڑوں کا احترام کرتے ہیں،\nان کی باتیں مانتے ہیں،\nان سے پیار کرتے ہیں،\nان کی عزت کرتے ہیں،\n\nہم سب کے دوست ہیں،\nہم سب سے محبت کرتے ہیں،\nہم سب کے ساتھ مل کر رہتے ہیں،\nہم سب کی مدد کرتے ہیں،\n\nہم بچے ہیں، ہم مستقبل ہیں،\nہماری امیدیں روشن ہیں،\nہمارے خواب بڑے ہیں،\nہم انہیں پورا کریں گے،\n\nہم بچے ہیں، ہم پاکستان ہیں،\nہم اسے ترقی دیں گے،\nاسے خوشحال کریں گے،\nاسے ایک عظیم ملک بنائیں گے،\n\nآؤ بچو، ہم سب مل کر،\nاپنا مستقبل روشن کریں،\nاپنے ملک کو ترقی دیں،\nاور اسے ایک عظیم ملک بنائیں۔'

#### chaining with LCEL

In [17]:
chain_1 = model | parser
chain_1.invoke(message)

'Hi, Asif. I am glad to meet you.'

In [18]:
chain_2 = model | parser
chain_1.invoke(message_2)

'**بچوں کی دنیا**\n\nچھوٹی سی دنیا، رنگ برنگی،\nکھیل کود اور ہنسی مذاقی،\nبچے معصوم، دل شاد و آباد،\nجنت کا ٹکڑا، دنیا کا سوغات۔\n\nپھولوں سے سجی، تتلیوں سے بھری،\nپرندوں کی چہچہاہٹ، مٹھی بھر خوشی،\nپریوں کی کہانی، خوابوں کی دنیا،\nبچوں کا جہاں، جہاں خوشی رہتی۔\n\nگڑیوں سے کھیلنا، پتنگ اڑانا،\nدوڑنا کودنا، ہلکی سی لڑائی،\nمسکراہٹ چہرے پر، آنکھوں میں چمک،\nبچپن کا لطف، زندگی کی بہارک۔\n\nسکول جانا، نئی چیزیں سیکھنا،\nدوست بنانا، علم کا خزانہ بھرنا،\nاساتذہ کی محبت، والدین کا پیار،\nبچوں کی دنیا، جہاں سب کچھ پیارا۔\n\nبڑے ہوں گے تو یاد آئے گا،\nیہ بچپن کا زمانہ، یہ لطف و بہار،\nبچوں کی دنیا، جنت کا ٹکڑا،\nایک خواب جیسا، یادوں کا خزانہ۔'

### Prompt templating

In [19]:
from langchain.prompts import ChatPromptTemplate

In [20]:
prompt_template = ChatPromptTemplate.from_messages([
    "system","translate the following into {language}",
    "user","{text}"]
)

In [21]:
prompt = prompt_template.invoke({"language":"italian","text":"GenAI is the future of tech we must all learn"})

In [22]:
print(prompt)

messages=[HumanMessage(content='system'), HumanMessage(content='translate the following into italian'), HumanMessage(content='user'), HumanMessage(content='GenAI is the future of tech we must all learn')]


In [23]:
response = parser.invoke(model.invoke(prompt))

In [24]:
response

'GenAI è il futuro della tecnologia che tutti dobbiamo imparare'

In [25]:
prompt_2= prompt_template.invoke({"language":"german","text":"GenAI is the future of tech we must all learn"})

In [26]:
parser.invoke(model.invoke(prompt_2))

'GenAI ist die Zukunft der Technologie, die wir alle lernen müssen'

## Applying chaining(LCEL)

In [27]:
chain_1 = prompt_template | model | parser

In [28]:
chain_1.invoke({"language":"pashto","text":"Aamir the great boy gonna get to married soon but he did not invite me so sad although he is my good friend i dont understand why he ignore me, i dont like this really"})

'امیر ستر زوی به ټه کې واده کوي خو ما ته یې د ننوتلو بلنه نه ده ورکړې، ډېر غمجن یم خو هغه زما ښه ملګری دی، ما نموږم ولې ما ته بې پام کوي، ما ته دا ډېره نه خوښېږي'

### Agents

In [29]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [31]:
memory = MemorySaver()

In [32]:
search = TavilySearchResults(max_results=2)

In [33]:
tools = [search]

In [39]:
agent_executor = create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [40]:
config = {"configurable":{"thread_id": "chat123"}}

In [46]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="you are helpful chatbot?")]},config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='Yes, I am a chatbot designed to help you with a variety of tasks and questions. I am still under development, but I am learning new things every day. How can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f749d641-6d8e-4831-9eee-d894d8bceacf-0', usage_metadata={'input_tokens': 2007, 'output_tokens': 40, 'total_tokens': 2047})]}}


In [47]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="who are you??")]},config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='I am a chatbot designed to help you with a variety of tasks and questions. I am still under development, but I am learning new things every day. How can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b94f642e-7036-4f0a-b8e7-c0fc0ce5cebe-0', usage_metadata={'input_tokens': 2053, 'output_tokens': 38, 'total_tokens': 2091})]}}


In [48]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="Hi , i am asif!")]},config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='Nice to meet you, Asif! How can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f91eb1d2-b9fe-4adc-8abd-dddfd7584dec-0', usage_metadata={'input_tokens': 2100, 'output_tokens': 15, 'total_tokens': 2115})]}}


In [50]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="I live in Dubai!")]},config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Dubai"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-14dc1ce8-b37a-470a-9670-657c3b556c2c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Dubai'}, 'id': 'b483968f-7648-46b4-9c62-ee7689bee113', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2856, 'output_tokens': 22, 'total_tokens': 2878})]}}
{'tools': {'messages': [ToolMessage(cont